## Sequential Quadratic Programming
1 a i

In [3]:
using FiniteDiff
using Optim
using ForwardDiff
using ForwardDiff: gradient, hessian, jacobian
using Random
using LinearAlgebra

# Define the Lagrangian function
function lagrangian(domain, a, b, r)
    x = domain[1:end-1]
    λ = domain[end]
    
    n = length(x)
    sum_terms = 0
    for i in 1:n-1
        sum_terms += a * (1 - x[i])^2 + b * (x[i+1] - x[i]^2)^2
    end
    
    constraint_term = 0
    for i in 1:n
        constraint_term += λ * (x[i]^2 - r)
    end
    
    return sum_terms + constraint_term
end

# Define the gradient of the Lagrangian using ForwardDiff
function gradient_lagrangian(domain, a, b, r)
    x = domain[1:end-1]
    λ = domain[end]
    
    ∇L_x = ForwardDiff.gradient(x -> lagrangian([x; λ], a, b, r), x)
    
    return ∇L_x
end

# Define the function to sample points uniformly on the unit square
function sample_points_uniformly(n_points, n_dimensions)
    points = rand(n_points, n_dimensions)
    return points
end

# Function to compute gradient using finite differences
function finite_diff_lagrangian(domain, a, b, r; ϵ=1e-15)
    x = domain[1:end-1]
    λ = domain[end]
    
    ∇L_x = FiniteDiff.finite_difference_gradient(x -> lagrangian([x; λ], a, b, r), x)
    
    return ∇L_x
end

function test_gradients(n, k, a, b, r)
    points = sample_points_uniformly(k, n)  # Sample points uniformly
    errors_fd = zeros(k)
    errors_forward_diff = zeros(k)

    for i in 1:k
        domain = points  # Append λ = 0 to the sampled point
        ∇L_x_fd = finite_diff_lagrangian(domain, a, b, r)

        ∇L_x_forward_diff = gradient_lagrangian(domain, a, b, r)
        
        # Calculate RMSE for finite differences
        errors_fd[i] = sqrt(mean((∇L_x_fd .- ∇L_x_forward_diff).^2))
        
        # Calculate RMSE for ForwardDiff
    end

    return errors_fd
end

using Statistics
# Test the gradients
n = 5
k = 100
a = 1
b_values = [1, 100, 1000]  # Different values of b to test

for b in b_values
    errors_fd, errors_forward_diff = test_gradients(n, k, a, b, n)
    println("Results for b = $b:")
    println("RMSE using finite differences: $(mean(errors_fd))")
end

Results for b = 1:


RMSE using finite differences: 2.30223222267748e-9
Results for b = 100:
RMSE using finite differences: 9.261150370712152e-8


Results for b = 1000:
RMSE using finite differences: 1.065152628847392e-6


1 a ii

In [4]:
# Implement Newton's very specifically for this lagrangian method
function newton(x0, a, b, r, λ; tol = 10e-6, itermax = 100, trace=false)
    x = copy(x0)
    

    err  = Inf
    iter = 0
    while ! (err < tol) && iter < itermax
        # Calculate the gradient and the Hessian of the Lagrangian
        fx  = lagrangian([x; λ], 1, 1, r)

        Df   = ForwardDiff.gradient(x -> lagrangian([x; λ], a, b, r), x)
        if Df == zeros(length(x0))
            return (;fx = lagrangian([x; λ], a, b, r), x, iter)
            break
        end

        D²f  = ForwardDiff.hessian(x -> lagrangian([x; λ], a, b, r), x)

        # search direction
        sk = -D²f \ Df

        # update x
        x′ = x + sk
        
        # Check for convergence
        err = norm(x′ - x)

        # copy x′ to be old x and start again
        x .= x′
        iter += 1

        trace && @info "Trace Information" iter x fx = lagrangian([x; λ], 1, 1, r)
    end

    return (;fx = lagrangian([x; λ], 1, 1, r), x, iter)
end

a=1
b=1
r=2
λ = 0
x_0 = zeros(2)
println(newton(x_0, a, b, r, λ ))
# x0 only seems to work with floats, so just being aware of this is useful

(fx = 0.0, x = [1.0, 1.0], iter = 2)


1 a iii

In [5]:
# Test different cases for initial guess values, when does it work, potentially changing λ to see if it has 
#effects. KEEPING r=2

1 a iv

In [6]:
# ALLOWED TO CHANGE r=2


## Penalty method
1 bi

In [15]:
using ForwardDiff
using Optim

a = 10
b = 10
r = 10
function Rosenbrock(x, a, b, r)
    sum = 0
    for i in 1:length(x)-1
        calc = a*(1-x[i])^2 + b*(x[i+1]-x[i])^2
        sum += calc
    end
    return sum
end

function constraint(x, r)
    sum = 0
    for i in 1:length(x)
        sum += x[i]^2
    end
    return sum - r
end

function solve_penalty(P, x0, a, b, r)
    # Set up the penalized objective 
    function objective(x)
        return Rosenbrock(x, a, b, r) + P * norm(constraint(x, r))^2
    end

    # Solve with Newton's method
    ret = optimize(objective, x0, LBFGS())

    # Keep things interpretable
    return ret
end

solve_penalty (generic function with 2 methods)

In [16]:
println(solve_penalty(1e12, zeros(10)))

 * Status: success

 * Candidate solution
    Final objective value:     1.000000e+14

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 0.00e+00 ≤ 0.0e+00
    |x - x'|/|x'|          = NaN ≰ 0.0e+00
    |f(x) - f(x')|         = NaN ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = NaN ≰ 0.0e+00
    |g(x)|                 = 0.00e+00 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    0
    f(x) calls:    1
    ∇f(x) calls:   1



When we have n=10 we set $x_0 = [0]*10$, we see that with the large value of the penalty, we congerge to $x_{i}\text{'s} = -1$ which because we know the true minimum is when $x_{i}\text{'s} = 1$ we know this is not the answer we are looking for.

1 b ii

In [17]:
function penalty_method(x0, a, b, r; tol = 1e-12, γ = 10, trace = false)
    iter = 0
    num_evals = 0
    P = 1
    x = copy(x0)
    while true
        # Solve with penalty parameter P
        ret = solve_penalty(P, x, a, b, r)

        # Keep track of the total number of function evals (really, Newton steps in this case)
        num_evals += ret.iterations

        # If P is large enough, we're done
        P > 1e12 && break

        # Otherwise we keep going, using the solution as a warmstart
        # Increment P up
        x .= ret.minimizer  # Update x with the minimizer found
        gx = constraint(x, r)
        P *= γ
        iter += 1

        trace && @info "Trace" iter P penalty_violation = gx
    end

    return (; fx = Rosenbrock(x, a, b, r), x, iter, num_evals)
end


penalty_method (generic function with 2 methods)

In [18]:
x0 = zeros(10)
penalty_ret = penalty_method(x0, tol=1e-12, trace=true)

┌ Info: Trace
│   iter = 1
│   P = 10
│   penalty_violation = -1.6370904631912708e-11
└ @ Main c:\Users\reube\OneDrive\Desktop\Econ_programming\Week_6\Attempt problem set 5.ipynb:24
┌ Info: Trace
│   iter = 2
│   P = 100
│   penalty_violation = -1.6370904631912708e-11
└ @ Main c:\Users\reube\OneDrive\Desktop\Econ_programming\Week_6\Attempt problem set 5.ipynb:24
┌ Info: Trace
│   iter = 3
│   P = 1000
│   penalty_violation = -1.6370904631912708e-11
└ @ Main c:\Users\reube\OneDrive\Desktop\Econ_programming\Week_6\Attempt problem set 5.ipynb:24
┌ Info: Trace
│   iter = 4
│   P = 10000
│   penalty_violation = -1.6370904631912708e-11
└ @ Main c:\Users\reube\OneDrive\Desktop\Econ_programming\Week_6\Attempt problem set 5.ipynb:24
┌ Info: Trace
│   iter = 5
│   P = 100000
│   penalty_violation = -1.6370904631912708e-11
└ @ Main c:\Users\reube\OneDrive\Desktop\Econ_programming\Week_6\Attempt problem set 5.ipynb:24
┌ Info: Trace
│   iter = 6
│   P = 1000000
│   penalty_violation = -1.6370904631

┌ Info: Trace
│   iter = 11
│   P = 100000000000
│   penalty_violation = -1.6370904631912708e-11
└ @ Main c:\Users\reube\OneDrive\Desktop\Econ_programming\Week_6\Attempt problem set 5.ipynb:24
┌ Info: Trace
│   iter = 12
│   P = 1000000000000
│   penalty_violation = -1.6370904631912708e-11
└ @ Main c:\Users\reube\OneDrive\Desktop\Econ_programming\Week_6\Attempt problem set 5.ipynb:24
┌ Info: Trace
│   iter = 13
│   P = 10000000000000
│   penalty_violation = -1.6370904631912708e-11
└ @ Main c:\Users\reube\OneDrive\Desktop\Econ_programming\Week_6\Attempt problem set 5.ipynb:24


(fx = 1.0261426364130259e-20, x = [0.9999999999898055, 0.9999999999897117, 1.0000000000014042, 0.9999999999989639, 1.0000000000071068, 0.9999999999948097, 1.0000000000080815, 0.9999999999988493, 0.9999999999982205, 1.0000000000048612], iter = 13, num_evals = 29)

When n=r the penalty constraint method works it out almost immediately, however this is due to the efficiency of the Newton method rather than the penalty method. When $n \neq r$ there is not the option for all the x's to be 1 as this would violate the constraint. So it converges towards a different solution.

In [11]:
x0 = 1*ones(10)
penalty_ret = penalty_method(x0, tol=1e-12, trace=true)

┌ Info: Trace
│   iter = 1
│   P = 10
│   penalty_violation = 0.0
└ @ Main c:\Users\reube\OneDrive\Desktop\Econ_programming\Week_6\Attempt problem set 5.ipynb:24
┌ Info: Trace
│   iter = 2
│   P = 100
│   penalty_violation = 0.0
└ @ Main c:\Users\reube\OneDrive\Desktop\Econ_programming\Week_6\Attempt problem set 5.ipynb:24
┌ Info: Trace
│   iter = 3
│   P = 1000
│   penalty_violation = 0.0
└ @ Main c:\Users\reube\OneDrive\Desktop\Econ_programming\Week_6\Attempt problem set 5.ipynb:24
┌ Info: Trace
│   iter = 4
│   P = 10000
│   penalty_violation = 0.0
└ @ Main c:\Users\reube\OneDrive\Desktop\Econ_programming\Week_6\Attempt problem set 5.ipynb:24
┌ Info: Trace
│   iter = 5
│   P = 100000
│   penalty_violation = 0.0
└ @ Main c:\Users\reube\OneDrive\Desktop\Econ_programming\Week_6\Attempt problem set 5.ipynb:24
┌ Info: Trace
│   iter = 6
│   P = 1000000
│   penalty_violation = 0.0
└ @ Main c:\Users\reube\OneDrive\Desktop\Econ_programming\Week_6\Attempt problem set 5.ipynb:24
┌ Info: Trace

(fx = 0.0, x = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], iter = 13, num_evals = 12)

## Augmented Lagrangian
1 c i

In [12]:
function augmented_lagrangian_inner(f, g, P, λ, x0)
    function objective(x)
        gx = g(x)
        f(x) + 0.5 * P * norm(gx)^2 + dot(λ, gx)
    end

    # Solve the augmented problem
    ret = optimize(objective, x0, LBFGS())
    return ret.minimizer
end

function augmented_lagrangian(f, g, x0; tol = 1e-8, γ = 10, trace=false)
    gx = g(x0)
    λ = ones(size(gx))  # Initialize λ as zeros
    P = 1.0
    x = copy(x0)
    num_evals = 0
    iter = 0

    while true 
        # Solve the inner problem 
        x′ = augmented_lagrangian_inner(f, g, P, λ, x)
        num_evals += 1
        iter += 1
        
        # Update λ
        gx = g(x′)
        λ′ = λ .+ gx .* P

        # Check if we're violating the constraints
        err = norm(λ′ .- λ)^2
        if err < tol
            break
        else # otherwise keep going
            P *= γ
            x .= x′
            λ .= λ′
            trace && @info "Trace" iter P err penalty_violation = gx
        end
    end

    return (fx = f(x), x, λ, num_evals, iter)
end


augmented_lagrangian (generic function with 1 method)

In [13]:
println(augmented_lagrangian(Rosenbrock, constraint, zeros(10), trace=true))

┌ Info: Trace
│   iter = 1
│   P = 10.0
│   err = 0.4946585212453418
│   penalty_violation = -0.7033196437220717
└ @ Main c:\Users\reube\OneDrive\Desktop\Econ_programming\Week_6\Attempt problem set 5.ipynb:38
┌ Info: Trace
│   iter = 2
│   P = 100.0
│   err = 0.08142201196563384
│   penalty_violation = -0.02853454256960042
└ @ Main c:\Users\reube\OneDrive\Desktop\Econ_programming\Week_6\Attempt problem set 5.ipynb:38
┌ Info: Trace
│   iter = 3
│   P = 1000.0
│   err = 0.00012746851424601653
│   penalty_violation = -0.00011290195491930888
└ @ Main c:\Users\reube\OneDrive\Desktop\Econ_programming\Week_6\Attempt problem set 5.ipynb:38


(fx = 2.525538859999929e-9, x = [0.9999955244115429, 0.9999955226802132, 0.9999955174862142, 0.9999955036355321, 0.9999954672774264, 0.9999953720536301, 0.9999951227399121, 0.9999944700214786, 0.9999927611772851, 0.9999882873561534], λ = 

fill(4.473508999325304e-5), num_evals = 4, iter = 4)


## Parameter Transformation
1 d i

In [14]:
function h(Z, r)
    n = length(Z) + 1
    X = zeros(n)

    exp_Z = exp.(Z)

    exp_sum = sum(exp_Z)

    for i in 1:n-1
        X[i] = (exp(Z[i]) / (1 + exp_sum))^(1/2)
    end
    X[n] = (1 / (1 + exp_sum))^(1/2)
    return sqrt(r) * X
end

function h_inv(X)
    n = length(X)
    Z = zeros(n-1)
    for i in 1:n-1
        Z[i] = 2 * log(X[i]/X[n])
    end
    return Z
end

function Rosenbrock(X, a, b)
    sum = 0
    for i in 1:length(X)-1
        sum += (a*(1-X[i])^2 + b*(X[i+1]-X[i]^2)^2)
    end
    return sum
end

function constraint(X, r)
    sum = 0
    for i in 1:length(X)
        sum += X[i]^2
    end
    return sum - r
end

function Rosenbrock_unconstrained(Z, a, b, r)
    X = h(Z, r)
    return Rosenbrock(X, a, b)
end

using Optim
a = 1
b = 1
r = 2
z0 = [5.0]

# Define the objective function to pass to Optim
objective_function(Z) = Rosenbrock_unconstrained(Z, a, b, r)

# Call the optimization routine
result = optimize(objective_function, z0, LBFGS())

 * Status: success

 * Candidate solution
    Final objective value:     7.039971e-18

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 1.48e-03 ≰ 0.0e+00
    |x - x'|/|x'|          = 4.42e+05 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.37e-06 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.95e+11 ≰ 0.0e+00
    |g(x)|                 = 4.19e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    4
    f(x) calls:    14
    ∇f(x) calls:   14
